In [2]:
import numpy as np
from scipy import io

import os
import shutil
from distutils.dir_util import copy_tree

In [3]:
root = '../dataset'
dataset = 'Penn_Action_Labels'
splits = ['train', 'val']


In [3]:
for type in splits:
    print(f'{type} set size: ')
    path = os.path.join(root, dataset, 'penn_action_labels', type)
    for classname in sorted(os.listdir(path)):
        print(classname + ': ' + str(len([filename for filename in os.listdir(os.path.join(path, classname))])))
    print('\n')

train set size: 
baseball_pitch: 103
baseball_swing: 113
bench_press: 69
bowling: 134
clean_and_jerk: 40
golf_swing: 87
jumping_jacks: 56
pullups: 98
pushups: 102
situp: 50
squats: 111
tennis_forehand: 79
tennis_serve: 115


val set size: 
baseball_pitch: 63
baseball_swing: 57
bench_press: 71
bowling: 85
clean_and_jerk: 42
golf_swing: 77
jumping_jacks: 56
pullups: 101
pushups: 106
situp: 50
squats: 115
tennis_forehand: 74
tennis_serve: 69




In [4]:
ori_dataset = 'Penn_Action'
for type in splits:
    os.makedirs(os.path.join(root, ori_dataset, type), exist_ok=True)

In [10]:
tmp = os.path.join(root, dataset, 'penn_action_labels', 'train')
os.makedirs(os.path.join(root, ori_dataset, 'notin'), exist_ok=True)
#os.makedirs(os.path.join(root, ori_dataset, 'notin_val'), exist_ok=True)
class_list = [classname for classname in sorted(os.listdir(path))]

for f in ['train', 'val', 'notin']:
    for classname in class_list: 
        os.makedirs(os.path.join(root, ori_dataset, f, classname), exist_ok=True) 

In [6]:
class_list

['baseball_pitch',
 'baseball_swing',
 'bench_press',
 'bowling',
 'clean_and_jerk',
 'golf_swing',
 'jumping_jacks',
 'pullups',
 'pushups',
 'situp',
 'squats',
 'tennis_forehand',
 'tennis_serve']

In [13]:
# Switch split with 'train' and 'val'
split = 'train'
ori_path = os.path.join(root, ori_dataset, split)
path = os.path.join(root, dataset, 'penn_action_labels', split)

for classname in sorted(os.listdir(path)):
    #os.makedirs(os.path.join(ori_path, classname), exist_ok=True)
    tmp_path = os.path.join(ori_path, classname)
    for filename in os.listdir(os.path.join(path, classname)):
        name = os.path.splitext(filename)[0]
        os.makedirs(os.path.join(tmp_path, name))
        copy_tree(os.path.join(root, ori_dataset, 'frames', name), os.path.join(tmp_path, name))
        print(f'Successfully copy from {ori_dataset}/frames/{name} to {ori_dataset}/{split}/{classname}')

Successfully copy from Penn_Action/frames/0101 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0071 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0068 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0004 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0008 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0091 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0137 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0160 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0099 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0082 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0066 to Penn_Action/train/baseball_pitch
Successfully copy from Penn_Action/frames/0131 to Penn_Action/train/baseball_pitch
Succ

In [14]:
dict = {}
dict = dict.fromkeys(class_list, [])
dict

{'baseball_pitch': [],
 'baseball_swing': [],
 'bench_press': [],
 'bowling': [],
 'clean_and_jerk': [],
 'golf_swing': [],
 'jumping_jacks': [],
 'pullups': [],
 'pushups': [],
 'situp': [],
 'squats': [],
 'tennis_forehand': [],
 'tennis_serve': []}

In [15]:
path = os.path.join(root, dataset, 'penn_action_labels')
for key in dict:
    train = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(path, 'train', key))]
    val = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(path, 'val', key))]
    dict[key] = train + val

In [18]:
len(dict['squats'])

226

In [19]:
count = 0
for value in dict.values():
    count += len(value)
print("Number of videos in Penn Action Dataset: 2326")
print(f"Number of videos in per-frame-labelled Penn Action Dataset: {count}")


Number of videos in Penn Action Dataset: 2326
Number of videos in per-frame-labelled Penn Action Dataset: 2123


In [20]:
ori_path = os.path.join(root, ori_dataset)
notin_list = []
for item in os.listdir(os.path.join(ori_path, 'frames')):
    check_item = all(item not in value for value in dict.values())
    if check_item == True:
        notin_list.append(item)

In [21]:
print(f"Number of videos not in per-frame-labelled Penn Action Dataset: {len(notin_list)}")

Number of videos not in per-frame-labelled Penn Action Dataset: 203


In [22]:
no_classname = ['jump_rope', 'strum_guitar']
final_notin_list = []
for item in notin_list:
    f = io.loadmat(os.path.join(ori_path, 'labels', item + '.mat'))
    if f['action'][0] not in no_classname:
        final_notin_list.append(item)

In [23]:
print(f'Number of videos not in per-frame-labelled Penn Action Dataset and \
    not in jump_rope & strum_guitar class {len(final_notin_list)}')

Number of videos not in per-frame-labelled Penn Action Dataset and     not in jump_rope & strum_guitar class 27


In [25]:
for item in final_notin_list:
    f = io.loadmat(os.path.join(ori_path, 'labels', item + '.mat'))
    _classname = f['action'][0]
    os.makedirs(os.path.join(root, ori_dataset, 'notin', _classname, item), exist_ok=True)
    copy_tree(os.path.join(root, ori_dataset, 'frames', item), \
        os.path.join(root, ori_dataset, 'notin', _classname, item))
    

In [43]:
# Check all files in notin folders
notin_path = os.path.join(root, ori_dataset, 'notin')
print(f"{'Action' :>15} {'No. files' :>10} {'Name file' :>50}")
for classname in sorted(os.listdir(notin_path)):
    print('{:>15} {:>10} {:>50}'.format(classname, len(os.listdir(os.path.join(notin_path, classname))), '{}'.format(os.listdir(os.path.join(notin_path, classname)))))
    #print(f'{classname} \t \t {len(os.listdir(os.path.join(notin_path, classname)))} \t {os.listdir(os.path.join(notin_path, classname))}')
    #for item in os.listdir(os.path.join(notin_path, classname)):
        

         Action  No. files                                          Name file
 baseball_pitch          1                                           ['0005']
 baseball_swing          3                           ['0260', '0217', '0199']
    bench_press          0                                                 []
           bowl          1                                           ['0557']
        bowling          0                                                 []
 clean_and_jerk          6   ['0722', '0782', '0716', '0707', '0704', '0756']
     golf_swing          2                                   ['0821', '0846']
  jumping_jacks          0                                                 []
        pullups          0                                                 []
         pushup          3                           ['1403', '1516', '1361']
        pushups          0                                                 []
          situp          0                                      

In [6]:
# Check all videos in Penn_Action_Images train set
path = os.path.join(root, 'Penn_Action', 'train')
for classname in sorted(os.listdir(path)):
    print(f'{classname}: {len(os.listdir(os.path.join(path, classname)))}')

baseball_pitch: 103
baseball_swing: 113
bench_press: 69
bowling: 134
clean_and_jerk: 40
golf_swing: 87
jumping_jacks: 56
pullups: 98
pushups: 102
situp: 50
squats: 111
tennis_forehand: 79
tennis_serve: 115


In [8]:
# Check all videos in Penn_Action_Images test set
path = os.path.join(root, 'Penn_Action', 'val')
for classname in sorted(os.listdir(path)):
    print(f'{classname}: {len(os.listdir(os.path.join(path, classname)))}')

baseball_pitch: 63
baseball_swing: 57
bench_press: 71
bowling: 85
clean_and_jerk: 42
golf_swing: 77
jumping_jacks: 56
pullups: 101
pushups: 106
situp: 50
squats: 115
tennis_forehand: 74
tennis_serve: 69


## Create video list for creating tfrecords

##### For training

In [7]:
video_path = os.path.join(root, 'Penn_Action', 'train')
filename = '%s.txt'
des_path = os.path.join(root, 'Penn_Action', 'files', 'train')

In [8]:
for classname in sorted(os.listdir(video_path)):
    f = open(os.path.join(des_path, filename % classname), 'w')
    paths = [os.path.join(classname, file) for file in sorted(os.listdir(os.path.join(video_path, classname)))]
    f.writelines('%s \n' % path for path in paths)
    f.close()       



##### For validation

In [4]:
video_path = os.path.join(root, 'Penn_Action', 'val')
filename = '%s.txt'
des_path = os.path.join(root, 'Penn_Action', 'files', 'val')

In [10]:
for classname in sorted(os.listdir(video_path)):
    f = open(os.path.join(des_path, filename % classname), 'w')
    paths = [os.path.join(classname, file) for file in sorted(os.listdir(os.path.join(video_path, classname)))]
    f.writelines('%s \n' % path for path in paths)
    f.close()       

In [11]:
for classname in os.listdir(video_path):
    os.makedirs(os.path.join('./tfrecords', classname), exist_ok=True)

In [5]:
for classname in sorted(os.listdir(video_path)):
    print(classname)

baseball_pitch
baseball_swing
bench_press
bowling
clean_and_jerk
golf_swing
jumping_jacks
pullups
pushups
situp
squats
tennis_forehand
tennis_serve


In [1]:
import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

2022-07-23 10:53:05.591330: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Default GPU Device: /device:GPU:0


2022-07-23 10:53:06.983958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-23 10:53:08.550828: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 6416 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5
2022-07-23 10:53:08.552033: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:1 with 14795 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5
2022-07-23 10:53:08.564018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /device:GPU:0 with 6416 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1